In [ ]:
import json
import itertools
import re
import os
import pandas as pd

In [ ]:
with open('data/ios_signatures.json', encoding='utf-8') as fh:
    signatures = json.load(fh)

In [ ]:
regexs = []
for signature in signatures:
    regexs.append(signature['regex'])

# taken from: https://github.com/Exodus-Privacy/exodus-core
compiled_tracker_signature = [re.compile(signature['regex'], flags=re.MULTILINE | re.UNICODE)
                                       for signature in signatures]

In [ ]:
# taken from: https://github.com/Exodus-Privacy/exodus-core
def detect_trackers(filename):
    with open(filename, 'r') as f:
        class_list = f.readlines()
    
    args = [(compiled_tracker_signature[index], tracker, class_list)
                for (index, tracker) in enumerate(signatures)]

    results = []

    def _detect_tracker(sig, tracker, class_list):
        for clazz in class_list:
            if sig.search(clazz):
                return tracker
        return None

    for res in itertools.starmap(_detect_tracker, args):
        if res:
            results.append(res)

    trackers = [t['id'] for t in results if t is not None]

    return trackers

folder = "ios_log/classes"
files = os.listdir(folder)

from multiprocessing import Pool
import tqdm

from pathlib import Path

def detect(filename):
    results = {}
    results['app'] = Path(filename).stem[0:-8]
    results['trackers'] = detect_trackers(folder + filename)
    return results

pool = Pool(processes=4)
trackers = {}
for result in tqdm.tqdm(pool.imap_unordered(detect, files), total=len(files)):
    trackers[result['app']] = result['trackers']
    
with open('data_processed/ios_trackers.json', 'w', encoding='utf-8') as f:
     json.dump(trackers, f)